# VideoTransformer

TimeSformer(https://arxiv.org/abs/2102.05095), ViViT(https://arxiv.org/abs/2103.15691)

Welcome to the demo notebook for VideoTransformer. We'll showcase the prediction result by the above pre-trained models.

## Preliminaries

This section contains initial setup. Run it first.

In [6]:
!pip3 install --user torch
!pip3 install --user torchvision
!pip3 install --user matplotlib
!pip3 install --user decord
!pip3 install --user einops
!pip3 install --user scikit-image
!pip3 install --user pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
!pip uninstall einops
!pip install einops

Found existing installation: einops 0.6.0
Uninstalling einops-0.6.0:
  Would remove:
    /usr/local/lib/python3.8/dist-packages/einops-0.6.0.dist-info/*
    /usr/local/lib/python3.8/dist-packages/einops/*
Proceed (Y/n)? y
  Successfully uninstalled einops-0.6.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached einops-0.6.0-py3-none-any.whl (41 kB)


In [8]:
import torch
import torch.nn as nn
import numpy as np
import einops

from einops import rearrange, reduce, repeat
from IPython.display import display

!git clone https://github.com/mx-mark/VideoTransformer-pytorch.git
%cd VideoTransformer-pytorch

!pip uninstall decord
!pip install decord
!pip uninstall pytorch-lightning
!pip install pytorch-lightning
!pip uninstall lightning_utilities
!pip install lightning_utilities
!pip uninstall torchmetrics
!pip install torchmetrics
!pip uninstall tensorboardX
!pip install tensorboardX

import data_transform as T
from dataset import DecordInit, load_annotation_data
from transformer import PatchEmbed, TransformerContainer, ClassificationHead

Cloning into 'VideoTransformer-pytorch'...
remote: Enumerating objects: 159, done.
remote: Counting objects: 100% (120/120), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 159 (delta 61), reused 100 (delta 48), pack-reused 39
Receiving objects: 100% (159/159), 4.26 MiB | 8.07 MiB/s, done.
Resolving deltas: 100% (69/69), done.
/content/VideoTransformer-pytorch/VideoTransformer-pytorch/VideoTransformer-pytorch
Found existing installation: decord 0.6.0
Uninstalling decord-0.6.0:
  Would remove:
    /usr/local/lib/python3.8/dist-packages/decord-0.6.0.dist-info/*
    /usr/local/lib/python3.8/dist-packages/decord.libs/libXau-b2e5323c.so.6.0.0
    /usr/local/lib/python3.8/dist-packages/decord.libs/libavcodec-bc50294c.so.58.35.100
    /usr/local/lib/python3.8/dist-packages/decord.libs/libavdevice-bf61e037.so.58.5.100
    /usr/local/lib/python3.8/dist-packages/decord.libs/libavfilter-1e2243e2.so.7.40.101
    /usr/local/lib/python3.8/dist-packages/decord.libs/libavformat-8b

Found existing installation: pytorch-lightning 1.8.6
Uninstalling pytorch-lightning-1.8.6:
  Would remove:
    /usr/local/lib/python3.8/dist-packages/lightning_lite/*
    /usr/local/lib/python3.8/dist-packages/pytorch_lightning-1.8.6.dist-info/*
    /usr/local/lib/python3.8/dist-packages/pytorch_lightning/*
Proceed (Y/n)? y
  Successfully uninstalled pytorch-lightning-1.8.6
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pytorch_lightning-1.8.6-py3-none-any.whl (800 kB)


y
Found existing installation: lightning-utilities 0.5.0
Uninstalling lightning-utilities-0.5.0:
  Would remove:
    /usr/local/lib/python3.8/dist-packages/lightning_utilities-0.5.0.dist-info/*
    /usr/local/lib/python3.8/dist-packages/lightning_utilities/*
Proceed (Y/n)?   Successfully uninstalled lightning-utilities-0.5.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached lightning_utilities-0.5.0-py3-none-any.whl (18 kB)
y


Found existing installation: torchmetrics 0.11.0
Uninstalling torchmetrics-0.11.0:
  Would remove:
    /usr/local/lib/python3.8/dist-packages/torchmetrics-0.11.0.dist-info/*
    /usr/local/lib/python3.8/dist-packages/torchmetrics/*
Proceed (Y/n)? y
  Successfully uninstalled torchmetrics-0.11.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached torchmetrics-0.11.0-py3-none-any.whl (512 kB)


Found existing installation: tensorboardX 2.5.1
Uninstalling tensorboardX-2.5.1:
  Would remove:
    /usr/local/lib/python3.8/dist-packages/tensorboardX-2.5.1.dist-info/*
    /usr/local/lib/python3.8/dist-packages/tensorboardX/*
Proceed (Y/n)? y
  Successfully uninstalled tensorboardX-2.5.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached tensorboardX-2.5.1-py2.py3-none-any.whl (125 kB)


### Note
Please firstly dowload the weights and move to the current path `./VideoTransformer-pytorch/`
1. TimeSformer-B pre-trained on K400 https://drive.google.com/file/d/1jLkS24jkpmakPi3e5J8KH3FOPv370zvo/view?usp=sharing
2. ViViT-B pre-trained on K400 from https://drive.google.com/file/d/1-JVhSN3QHKUOLkXLWXWn5drdvKn0gPll/view?usp=sharing

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Video Transformer Model

We here load the pretrained weights of the transformer model TimeSformer-B or ViViT-B.

In [10]:
class TimeSformer(nn.Module):
    """TimeSformer. A PyTorch impl of `Is Space-Time Attention All You Need for
    Video Understanding? <https://arxiv.org/abs/2102.05095>`_

    Args:
        num_frames (int): Number of frames in the video.
        img_size (int | tuple): Size of input image.
        patch_size (int): Size of one patch.
        pretrained (str | None): Name of pretrained model. Default: None.
        embed_dims (int): Dimensions of embedding. Defaults to 768.
        num_heads (int): Number of parallel attention heads in
            TransformerCoder. Defaults to 12.
        num_transformer_layers (int): Number of transformer layers. Defaults to
            12.
        in_channels (int): Channel num of input features. Defaults to 3.
        dropout_p (float): Probability of dropout layer. Defaults to 0.
        conv_type (str): Type of the convolution in PatchEmbed layer. Defaults to Conv2d.
        attention_type (str): Type of attentions in TransformerCoder. Choices
            are 'divided_space_time', 'space_only' and 'joint_space_time'.
            Defaults to 'divided_space_time'.
        norm_layer (dict): Config for norm layers. Defaults to nn.LayerNorm.
        return_cls_token (bool): Whether to use cls_token to predict class label.
    """
    supported_attention_types = [
        'divided_space_time', 'space_only', 'joint_space_time'
    ]

    def __init__(self,
                 num_frames,
                 img_size,
                 patch_size,
                 embed_dims=768,
                 num_heads=12,
                 num_transformer_layers=12,
                 in_channels=3,
                 conv_type='Conv2d',
                 dropout_p=0.,
                 attention_type='divided_space_time',
                 norm_layer=nn.LayerNorm,
                 return_cls_token=True,
                 **kwargs):
        super().__init__()
        assert attention_type in self.supported_attention_types, (
            f'Unsupported Attention Type {attention_type}!')

        self.num_frames = num_frames
        self.embed_dims = embed_dims
        self.num_transformer_layers = num_transformer_layers
        self.attention_type = attention_type
        self.conv_type = conv_type
        self.return_cls_token = return_cls_token

        #tokenize & position embedding
        self.patch_embed = PatchEmbed(
            img_size=img_size,
            patch_size=patch_size,
            in_channels=in_channels,
            embed_dims=embed_dims,
            conv_type=conv_type)
        num_patches = self.patch_embed.num_patches

        # Divided Space Time Attention
        operator_order = ['time_attn','space_attn','ffn']
        container = TransformerContainer(
            num_transformer_layers=num_transformer_layers,
            embed_dims=embed_dims,
            num_heads=num_heads,
            num_frames=num_frames,
            norm_layer=norm_layer,
            hidden_channels=embed_dims*4,
            operator_order=operator_order)

        self.transformer_layers = container
        self.norm = norm_layer(embed_dims, eps=1e-6)

        self.cls_token = nn.Parameter(torch.zeros(1,1,embed_dims))
        num_patches = num_patches + 1

        # spatial pos_emb
        self.pos_embed = nn.Parameter(torch.zeros(1,num_patches,embed_dims))
        self.drop_after_pos = nn.Dropout(p=dropout_p)

        # temporal pos_emb
        self.time_embed = nn.Parameter(torch.zeros(1,num_frames,embed_dims))
        self.drop_after_time = nn.Dropout(p=dropout_p)

    def interpolate_pos_encoding(self, x, w, h):
        npatch = x.shape[1] - 1
        N = self.pos_embed.shape[1] - 1
        if npatch == N and w == h:
            return self.pos_embed
        class_pos_embed = self.pos_embed[:, 0]
        patch_pos_embed = self.pos_embed[:, 1:]
        dim = x.shape[-1]
        w0 = w // self.patch_embed.patch_size[0]
        h0 = h // self.patch_embed.patch_size[0]
        # we add a small number to avoid floating point error in the interpolation
        # see discussion at https://github.com/facebookresearch/dino/issues/8
        w0, h0 = w0 + 0.1, h0 + 0.1
        patch_pos_embed = nn.functional.interpolate(
            patch_pos_embed.reshape(1, int(math.sqrt(N)), int(math.sqrt(N)), dim).permute(0, 3, 1, 2),
            scale_factor=(w0 / math.sqrt(N), h0 / math.sqrt(N)),
            mode='bicubic',
        )
        assert int(w0) == patch_pos_embed.shape[-2] and int(h0) == patch_pos_embed.shape[-1]
        patch_pos_embed = patch_pos_embed.permute(0, 2, 3, 1).view(1, -1, dim)
        return torch.cat((class_pos_embed.unsqueeze(0), patch_pos_embed), dim=1)

    def forward(self, x):
        #Tokenize
        b, t, c, h, w = x.shape
        x = self.patch_embed(x)

        # Add Position Embedding
        cls_tokens = repeat(self.cls_token, 'b ... -> (repeat b) ...', repeat=x.shape[0])
        x = torch.cat((cls_tokens, x), dim=1)
        x = x + self.interpolate_pos_encoding(x, w, h) #self.pos_embed
        x = self.drop_after_pos(x)

        # Add Time Embedding
        cls_tokens = x[:b, 0, :].unsqueeze(1)
        x = rearrange(x[:, 1:, :], '(b t) p d -> (b p) t d', b=b)
        x = x + self.time_embed
        x = rearrange(x, '(b p) t d -> b (p t) d', b=b)
        x = torch.cat((cls_tokens, x), dim=1)
        x = self.drop_after_time(x)

        # Video transformer forward
        x = self.transformer_layers(x)

        x = self.norm(x)
        # Return Class Token
        if self.return_cls_token:
            return x[:, 0]
        else:
            return x[:, 1:].mean(1)

In [11]:
class ViViT(nn.Module):
    """ViViT. A PyTorch impl of `ViViT: A Video Vision Transformer`
        <https://arxiv.org/abs/2103.15691>

    Args:
        num_frames (int): Number of frames in the video.
        img_size (int | tuple): Size of input image.
        patch_size (int): Size of one patch.
        pretrained (str | None): Name of pretrained model. Default: None.
        embed_dims (int): Dimensions of embedding. Defaults to 768.
        num_heads (int): Number of parallel attention heads. Defaults to 12.
        num_transformer_layers (int): Number of transformer layers. Defaults to 12.
        in_channels (int): Channel num of input features. Defaults to 3.
        dropout_p (float): Probability of dropout layer. Defaults to 0..
        tube_size (int): Dimension of the kernel size in Conv3d. Defaults to 2.
        conv_type (str): Type of the convolution in PatchEmbed layer. Defaults to Conv3d.
        attention_type (str): Type of attentions in TransformerCoder. Choices
            are 'divided_space_time', 'fact_encoder' and 'joint_space_time'.
            Defaults to 'fact_encoder'.
        norm_layer (dict): Config for norm layers. Defaults to nn.LayerNorm.
        copy_strategy (str): Copy or Initial to zero towards the new additional layer.
        extend_strategy (str): How to initialize the weights of Conv3d from pre-trained Conv2d.
        use_learnable_pos_emb (bool): Whether to use learnable position embeddings.
        return_cls_token (bool): Whether to use cls_token to predict class label.
    """
    supported_attention_types = [
        'fact_encoder', 'joint_space_time', 'divided_space_time'
    ]

    def __init__(self,
                 num_frames,
                 img_size,
                 patch_size,
                 embed_dims=768,
                 num_heads=12,
                 num_transformer_layers=12,
                 in_channels=3,
                 dropout_p=0.,
                 tube_size=2,
                 conv_type='Conv3d',
                 attention_type='fact_encoder',
                 norm_layer=nn.LayerNorm,
                 return_cls_token=True,
                 **kwargs):
        super().__init__()
        assert attention_type in self.supported_attention_types, (
            f'Unsupported Attention Type {attention_type}!')

        num_frames = num_frames//tube_size
        self.num_frames = num_frames
        self.embed_dims = embed_dims
        self.num_transformer_layers = num_transformer_layers
        self.attention_type = attention_type
        self.conv_type = conv_type
        self.tube_size = tube_size
        self.num_time_transformer_layers = 4
        self.return_cls_token = return_cls_token

        #tokenize & position embedding
        self.patch_embed = PatchEmbed(
            img_size=img_size,
            patch_size=patch_size,
            in_channels=in_channels,
            embed_dims=embed_dims,
            tube_size=tube_size,
            conv_type=conv_type)
        num_patches = self.patch_embed.num_patches

        # Divided Space Time Transformer Encoder - Model 2
        transformer_layers = nn.ModuleList([])

        spatial_transformer = TransformerContainer(
            num_transformer_layers=num_transformer_layers,
            embed_dims=embed_dims,
            num_heads=num_heads,
            num_frames=num_frames,
            norm_layer=norm_layer,
            hidden_channels=embed_dims*4,
            operator_order=['self_attn','ffn'])

        temporal_transformer = TransformerContainer(
            num_transformer_layers=self.num_time_transformer_layers,
            embed_dims=embed_dims,
            num_heads=num_heads,
            num_frames=num_frames,
            norm_layer=norm_layer,
            hidden_channels=embed_dims*4,
            operator_order=['self_attn','ffn'])

        transformer_layers.append(spatial_transformer)
        transformer_layers.append(temporal_transformer)

        self.transformer_layers = transformer_layers
        self.norm = norm_layer(embed_dims, eps=1e-6)

        self.cls_token = nn.Parameter(torch.zeros(1,1,embed_dims))
        # whether to add one cls_token in temporal pos_enb
        num_frames = num_frames + 1
        num_patches = num_patches + 1

        self.pos_embed = nn.Parameter(torch.zeros(1,num_patches,embed_dims))
        self.time_embed = nn.Parameter(torch.zeros(1,num_frames,embed_dims))
        self.drop_after_pos = nn.Dropout(p=dropout_p)
        self.drop_after_time = nn.Dropout(p=dropout_p)
    
    def interpolate_pos_encoding(self, x, w, h):
        npatch = x.shape[1] - 1
        N = self.pos_embed.shape[1] - 1
        if npatch == N and w == h:
            return self.pos_embed
        class_pos_embed = self.pos_embed[:, 0]
        patch_pos_embed = self.pos_embed[:, 1:]
        dim = x.shape[-1]
        w0 = w // self.patch_embed.patch_size[0]
        h0 = h // self.patch_embed.patch_size[0]
        # we add a small number to avoid floating point error in the interpolation
        # see discussion at https://github.com/facebookresearch/dino/issues/8
        w0, h0 = w0 + 0.1, h0 + 0.1
        patch_pos_embed = nn.functional.interpolate(
            patch_pos_embed.reshape(1, int(math.sqrt(N)), int(math.sqrt(N)), dim).permute(0, 3, 1, 2),
            scale_factor=(w0 / math.sqrt(N), h0 / math.sqrt(N)),
            mode='bicubic',
        )
        assert int(w0) == patch_pos_embed.shape[-2] and int(h0) == patch_pos_embed.shape[-1]
        patch_pos_embed = patch_pos_embed.permute(0, 2, 3, 1).view(1, -1, dim)
        return torch.cat((class_pos_embed.unsqueeze(0), patch_pos_embed), dim=1)

    def forward(self, x):
        #Tokenize
        b, t, c, h, w = x.shape
        x = self.patch_embed(x)

        # Add Position Embedding
        cls_tokens = repeat(self.cls_token, 'b ... -> (repeat b) ...', repeat=x.shape[0])
        x = torch.cat((cls_tokens, x), dim=1)
        x = x + self.interpolate_pos_encoding(x, w, h)
        x = self.drop_after_pos(x)

        # fact encoder - CRNN style
        spatial_transformer, temporal_transformer, = *self.transformer_layers,
        x = spatial_transformer(x)

        # Add Time Embedding
        cls_tokens = x[:b, 0, :].unsqueeze(1)
        x = rearrange(x[:, 1:, :], '(b t) p d -> b t p d', b=b)
        x = reduce(x, 'b t p d -> b t d', 'mean')
        x = torch.cat((cls_tokens, x), dim=1)
        x = x + self.time_embed
        x = self.drop_after_time(x)

        x = temporal_transformer(x)

        x = self.norm(x)
        # Return Class Token
        if self.return_cls_token:
            return x[:, 0]
        else:
            return x[:, 1:].mean(1)

In [12]:
def replace_state_dict(state_dict):
	for old_key in list(state_dict.keys()):
		if old_key.startswith('model'):
			new_key = old_key[6:]
			state_dict[new_key] = state_dict.pop(old_key)
		else:
			new_key = old_key[9:]
			state_dict[new_key] = state_dict.pop(old_key)

In [13]:
def init_from_pretrain_(module, pretrained, init_module):
    if torch.cuda.is_available():
        state_dict = torch.load(pretrained)
    else:
        state_dict = torch.load(pretrained, map_location=torch.device('cpu'))
    if init_module == 'transformer':
        replace_state_dict(state_dict)
    elif init_module == 'cls_head':
        replace_state_dict(state_dict)
    else:
        raise TypeError(f'pretrained weights do not include the {init_module} module')
    msg = module.load_state_dict(state_dict, strict=False)
    return msg

In [24]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
num_frames = 8
frame_interval = 32
num_class = 400
arch = 'vivit' # turn to vivit for initializing vivit model

if arch == 'timesformer':
    pretrain_pth = '/content/drive/My Drive/vivit_model.pth'
    model = TimeSformer(num_frames=num_frames,
                        img_size=224,
                        patch_size=16,
                        embed_dims=768,
                        in_channels=3,
                        attention_type='divided_space_time',
                        return_cls_token=True)
elif arch == 'vivit':
    pretrain_pth = '/content/drive/My Drive/vivit_model.pth'
    num_frames = num_frames * 2
    frame_interval = frame_interval // 2
    model = ViViT(num_frames=num_frames,
                  img_size=224,
                  patch_size=16,
                  embed_dims=768,
                  in_channels=3,
                  attention_type='fact_encoder',
                  return_cls_token=True)
else:
    raise TypeError(f'not supported arch type {arch}, chosen in (timesformer, vivit)')

cls_head = ClassificationHead(num_classes=num_class, in_channels=768)
msg_trans = init_from_pretrain_(model, pretrain_pth, init_module='transformer')
msg_cls = init_from_pretrain_(cls_head, pretrain_pth, init_module='cls_head')
model.eval()
cls_head.eval()
model = model.to(device)
cls_head = cls_head.to(device)
print(f'load model finished, the missing key of transformer is:{msg_trans[0]}, cls is:{msg_cls[0]}')

load model finished, the missing key of transformer is:['transformer_layers.0.layers.0.attentions.0.attn.qkv.weight', 'transformer_layers.0.layers.0.attentions.0.attn.qkv.bias', 'transformer_layers.0.layers.0.attentions.0.attn.proj.weight', 'transformer_layers.0.layers.0.attentions.0.attn.proj.bias', 'transformer_layers.0.layers.1.attentions.0.attn.qkv.weight', 'transformer_layers.0.layers.1.attentions.0.attn.qkv.bias', 'transformer_layers.0.layers.1.attentions.0.attn.proj.weight', 'transformer_layers.0.layers.1.attentions.0.attn.proj.bias', 'transformer_layers.0.layers.2.attentions.0.attn.qkv.weight', 'transformer_layers.0.layers.2.attentions.0.attn.qkv.bias', 'transformer_layers.0.layers.2.attentions.0.attn.proj.weight', 'transformer_layers.0.layers.2.attentions.0.attn.proj.bias', 'transformer_layers.0.layers.3.attentions.0.attn.qkv.weight', 'transformer_layers.0.layers.3.attentions.0.attn.qkv.bias', 'transformer_layers.0.layers.3.attentions.0.attn.proj.weight', 'transformer_layers.0

## Data preprocess

Here we show the video demo and transform the video input for the model processing.

In [25]:
from IPython.display import display, HTML

video_path = './demo/YABnJL_bDzw.mp4'
html_str = '''
<video controls width=\"480\" height=\"480\" src=\"{}\">animation</video>
'''.format(video_path)
display(HTML(html_str))

In [26]:
# Prepare data preprocess
mean, std = (0.45, 0.45, 0.45), (0.225, 0.225, 0.225)
data_transform = T.Compose([
        T.Resize(scale_range=(-1, 256)),
        T.ThreeCrop(size=224),
        T.ToTensor(),
        T.Normalize(mean, std)
        ])
temporal_sample = T.TemporalRandomCrop(num_frames*frame_interval)

# Sampling video frames
video_decoder = DecordInit()
v_reader = video_decoder(video_path)
total_frames = len(v_reader)
start_frame_ind, end_frame_ind = temporal_sample(total_frames)
if end_frame_ind-start_frame_ind < num_frames:
    raise ValueError(f'the total frames of the video {video_path} is less than {num_frames}')
frame_indice = np.linspace(0, end_frame_ind-start_frame_ind-1, num_frames, dtype=int)
video = v_reader.get_batch(frame_indice).asnumpy()
del v_reader

video = torch.from_numpy(video).permute(0,3,1,2) # Video transform: T C H W
data_transform.randomize_parameters()
video = data_transform(video)
video = video.to(device)

## Video Classification

Here we use the pre-trained video transformer to classify the input video.

In [27]:
# Predict class label
with torch.no_grad():
    logits = model(video)
    output = cls_head(logits)
    output = output.view(3, 400).mean(0)
    cls_pred = output.argmax().item()

class_map = './k400_classmap.json'
class_map = load_annotation_data(class_map)
for key, value in class_map.items():
    if int(value) == int(cls_pred):
        print(f'the shape of ouptut: {output.shape}, and the prediction is: {key}')
        break

the shape of ouptut: torch.Size([400]), and the prediction is: finger_snapping
